<h2>Write a PySpark program to find the winner in each group.
The winner is each group is the player who scored the maximum total points within the group. In the case of tie, the lowest player_id wins.</h2>

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName(test).getOrCreate()

In [47]:
data=[(15,1),(25,1),(30,1),(45,1),(10,2),(35,2),(50,2)
     ,(20,3),(40,3)
     ]
schema="player_id int,group_id int"

player_df=spark.createDataFrame(data,schema)

data=[(1,15,45,3,0),
     (2,30,25,1,2),
      (3,30,15,2,0),
      (4,40,20,5,2),
      (5,35,50,1,1)
     ]
schema="""match_id int,
first_player int,
second_player int,
first_score int,
second_score int"""

matches_df=spark.createDataFrame(data,schema)

In [64]:
from pyspark.sql.functions import sum,avg,max
player1_score_df=matches_df.groupBy(matches_df.first_player).agg(sum(matches_df.first_score).alias("total_score")).select(matches_df.first_player.alias("player_id"),"total_score")
player2_score_df=matches_df.groupBy(matches_df.second_player).agg(sum(matches_df.second_score).alias("total_score")).select(matches_df.second_player.alias("player_id"),"total_score")

player_score=player1_score_df.unionAll(player2_score_df)

In [81]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
combined_df=player_score.join(player_df,player_score.player_id==player_df.player_id,how="inner").select(player_df.player_id,player_df.group_id,player_score.total_score)

window_spec=Window.partitionBy(combined_df.group_id).orderBy(combined_df.total_score.desc(),combined_df.player_id.asc())

combined_df_ranked=combined_df.select(row_number().over(window_spec).alias('rn'),combined_df.player_id,combined_df.group_id,combined_df.total_score).\
filter("rn==1").drop("rn").show()


+---------+--------+-----------+
|player_id|group_id|total_score|
+---------+--------+-----------+
|       15|       1|          3|
|       35|       2|          1|
|       40|       3|          5|
+---------+--------+-----------+

